# Asset Pricing — Week 3
## Long-Run Risks, Disaster Risk, and the Factor Zoo

---

Week 2 ended with a precise diagnosis: the CCAPM fails because aggregate consumption is too smooth to generate the observed equity premium at plausible risk aversion. Two structural remedies were introduced — habit formation and Epstein-Zin preferences. But habit formation requires extreme effective risk aversion in downturns, and Epstein-Zin under i.i.d. consumption growth does not resolve the equity premium puzzle.

This week develops two of the most quantitatively successful equilibrium models in the literature, and then transitions from equilibrium theory to the empirical cross-section of expected returns.

**Part A — Long-Run Risks (Bansal-Yaron, 2004)**: Combines Epstein-Zin preferences with a consumption process containing small, persistent components and stochastic volatility. The IES greater than one makes agents sensitive to long-horizon news, amplifying risk premia without extreme risk aversion.

**Part B — Disaster Risk (Rietz, 1988; Barro, 2006)**: Introduces rare, large consumption disasters that are nearly invisible in postwar US data but whose risk is priced. A small probability of catastrophic loss can generate large equity premia with modest risk aversion.

**Part C — The Arbitrage Pricing Theory and Factor Models**: Moves from equilibrium theory to empirical factor models. The APT provides a no-arbitrage foundation; the Fama-French models provide empirical implementations. We develop the tools for evaluating whether a factor model spans the mean-variance frontier.

**Prerequisites**: Weeks 1 and 2. The material in Part A requires comfort with lognormal moment generating functions and the Epstein-Zin SDF derived in Week 2.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.optimize import minimize, brentq
from scipy.stats import norm, t as t_dist
from scipy.linalg import solve, cholesky

plt.rcParams.update({
    'figure.dpi': 120,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
})

rng = np.random.default_rng(seed=314)

---

# Part A — Long-Run Risks

---

## 1. The Core Insight: Why Long-Run Risk Matters

Recall from Week 2 that the Epstein-Zin SDF is:
$$
M_{t+1} = \beta^{\theta} \left(\frac{C_{t+1}}{C_t}\right)^{-\theta/\psi} R_{w,t+1}^{\theta-1},
\quad \theta \equiv \frac{1-\gamma}{1-1/\psi}.
$$
When $\psi > 1$ (high IES) and $\gamma > 1$, we have $\theta < 0$. The SDF is *decreasing* in the wealth return $R_{w,t+1}$. This is the critical mechanism: when the wealth portfolio falls (bad news), $M$ rises, generating risk premia on assets that covary negatively with wealth.

Under i.i.d. consumption growth, the wealth return $R_w$ is simply a scaled version of consumption growth. The SDF then reduces to something close to the CRRA SDF — and we showed in Week 2 that this does not resolve the equity premium puzzle.

**The long-run risks idea (Bansal and Yaron, 2004)**: Suppose consumption growth contains a small but *highly persistent* component. Even if this component is tiny (undetectable in short samples), it generates large variation in the wealth portfolio return — because the wealth portfolio prices all future consumption. An agent with EZ preferences and $\psi > 1$ cares about this long-horizon risk, and demands a large premium for bearing it.

The analogy is useful: a small leak in a boat that will accumulate over 30 years is trivially undetectable day-to-day, but the long-run consequences are enormous. An agent who thinks far ahead prices this risk heavily.

---

## 2. The Bansal-Yaron (2004) Model

### 2.1 The Consumption and Dividend Process

Log consumption growth $g_{t+1} = \ln(C_{t+1}/C_t)$ and log dividend growth $g^d_{t+1} = \ln(D_{t+1}/D_t)$ follow:

$$
g_{t+1} = \mu_c + x_t + \sigma_t \eta_{t+1}
$$
$$
g^d_{t+1} = \mu_d + \phi x_t + \varphi \sigma_t u_{t+1}
$$
$$
x_{t+1} = \rho x_t + \phi_e \sigma_t e_{t+1}
$$
$$
\sigma^2_{t+1} = \bar{\sigma}^2 + \nu_1 (\sigma^2_t - \bar{\sigma}^2) + \sigma_w w_{t+1}
$$

where $\eta_{t+1}$, $u_{t+1}$, $e_{t+1}$, $w_{t+1}$ are i.i.d. $\mathcal{N}(0,1)$ shocks, independent of each other.

**State variables**:
- $x_t$: the **long-run component** of consumption growth. It follows an AR(1) with persistence $\rho \approx 0.979$ (annual). Its variance is tiny relative to the short-run shock $\sigma_t \eta_{t+1}$, but because $\rho$ is close to one, it has a large cumulative effect on the level of consumption.
- $\sigma^2_t$: **stochastic volatility** of consumption growth. When $\sigma_t$ is high, both consumption shocks and long-run risk shocks are amplified.

**Key parameters**:
- $\phi > 1$: dividends are levered consumption. A 1% shock to $x_t$ produces a $\phi\%$ shock to dividend growth. This leverage amplifies risk premia on the dividend claim (equity) relative to the consumption claim.
- $\varphi > 1$: dividends are also more volatile than consumption (additional idiosyncratic volatility).

### 2.2 Why Long-Run Risk Is Priced

The wealth portfolio $R_w$ prices the claim to all future consumption. When $x_t$ rises, the market revises *all future* expected consumption growth upward. The value of the wealth portfolio rises substantially — much more than would be warranted by the current-period shock alone.

Under $\psi > 1$: the agent prefers early resolution of uncertainty (EZ with $\gamma > 1/\psi$ values knowing bad news early). News about the persistent component $x_t$ therefore directly enters the SDF through $R_w$. The equity premium has two components:

$$
E[R^e] - R_f = \underbrace{\text{(short-run risk premium)}}_{\propto \text{Cov}(g_t, r^e_t)} + \underbrace{\text{(long-run risk premium)}}_{\propto \text{Cov}(x_t, r^e_t) \cdot \text{(persistence loading)}}.
$$

The long-run risk premium can be large even when the short-run covariance between consumption growth and equity returns is small — resolving the Mehra-Prescott finding.

In [ ]:
# Bansal-Yaron (2004) calibration (monthly, Table 1)
# Parameters
mu_c    = 0.0015    # mean monthly consumption growth
mu_d    = 0.0015    # mean monthly dividend growth
rho     = 0.979**(1/12)  # monthly persistence of x (annualised 0.979)
phi_e   = 0.044     # volatility of shock to x, scaled by sigma
phi_d   = 3.0       # leverage (dividend loading on x)
varphi  = 4.5       # extra dividend vol
nu1     = 0.987**(1/12)  # monthly persistence of sigma^2
sigma_bar = np.sqrt(0.0072**2)  # unconditional monthly std of consumption growth
sigma_w   = 0.23e-5  # vol of vol shock

# Preferences
gamma_by = 10.0
psi_by   = 1.5
beta_by  = 0.998   # monthly
theta_by = (1 - gamma_by) / (1 - 1/psi_by)

T_by = 12 * 300   # 300 years monthly

# Simulate the state variables
eta  = rng.standard_normal(T_by)
e    = rng.standard_normal(T_by)
u    = rng.standard_normal(T_by)
w    = rng.standard_normal(T_by)

x      = np.empty(T_by + 1)
sig2   = np.empty(T_by + 1)
x[0]   = 0.0
sig2[0] = sigma_bar**2

g_c = np.empty(T_by)   # consumption growth
g_d = np.empty(T_by)   # dividend growth

for t in range(T_by):
    sig = np.sqrt(max(sig2[t], 1e-8))
    g_c[t] = mu_c + x[t] + sig * eta[t]
    g_d[t] = mu_d + phi_d * x[t] + varphi * sig * u[t]
    x[t+1]    = rho * x[t] + phi_e * sig * e[t]
    sig2[t+1] = sigma_bar**2 + nu1 * (sig2[t] - sigma_bar**2) + sigma_w * w[t]
    sig2[t+1] = max(sig2[t+1], 1e-8)

sig_arr = np.sqrt(sig2[:-1])

# Annualise moments for inspection
g_c_ann = np.array([g_c[i*12:(i+1)*12].sum() for i in range(T_by//12)])
g_d_ann = np.array([g_d[i*12:(i+1)*12].sum() for i in range(T_by//12)])
x_ann   = x[::12]

print("=" * 60)
print("Simulated Bansal-Yaron Process — Annual Moments")
print("=" * 60)
print(f"  E[g_c]          = {g_c_ann.mean()*100:.3f}%   (data: ~1.8%)")
print(f"  std(g_c)        = {g_c_ann.std()*100:.3f}%   (data: ~2.7%)")
print(f"  AC(1) of g_c    = {np.corrcoef(g_c_ann[:-1], g_c_ann[1:])[0,1]:.3f}   (near 0 in data)")
print(f"  E[g_d]          = {g_d_ann.mean()*100:.3f}%")
print(f"  std(g_d)        = {g_d_ann.std()*100:.3f}%   (data: ~11%)")
print(f"  Corr(g_c, g_d)  = {np.corrcoef(g_c_ann, g_d_ann)[0,1]:.3f}")
print(f"  std(x_t) annual = {x_ann.std()*100:.3f}%   (tiny relative to g_c shocks)")

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

years = np.arange(T_by // 12)

ax = axes[0, 0]
ax.plot(years[:100], g_c_ann[:100] * 100, lw=1.2, color='steelblue')
ax.set_xlabel('Year'); ax.set_ylabel('Log consumption growth (%)')
ax.set_title('Consumption Growth\n(first 100 years)')

ax2 = axes[0, 1]
ax2.plot(years[:100], x_ann[:100] * 1000, lw=1.5, color='#e74c3c')
ax2.axhline(0, color='black', lw=0.8, ls='--')
ax2.set_xlabel('Year'); ax2.set_ylabel('Long-run component $x_t$ (×1000)')
ax2.set_title('Long-Run Component $x_t$\n(persistent, tiny in magnitude)')

ax3 = axes[1, 0]
ax3.plot(years[:100], g_d_ann[:100] * 100, lw=1.0, color='#27ae60', alpha=0.8)
ax3.set_xlabel('Year'); ax3.set_ylabel('Log dividend growth (%)')
ax3.set_title('Dividend Growth\n(levered, more volatile than consumption)')

ax4 = axes[1, 1]
sig2_ann = np.array([sig2[i*12] for i in range(T_by//12)])
ax4.plot(years[:100], np.sqrt(sig2_ann[:100]) * 100 * np.sqrt(12), lw=1.5, color='#8e44ad')
ax4.axhline(sigma_bar * 100 * np.sqrt(12), color='black', lw=1, ls='--',
            label=f'Unconditional σ = {sigma_bar*100*np.sqrt(12):.2f}% p.a.')
ax4.set_xlabel('Year'); ax4.set_ylabel('Annualised consumption vol (%)')
ax4.set_title('Stochastic Volatility $\\sigma_t$')
ax4.legend(fontsize=9)

plt.tight_layout()
plt.suptitle('Bansal-Yaron (2004) State Variables', y=1.01, fontsize=13, fontweight='bold')
plt.show()

### 2.3 Log-Linearisation and Analytical Results

The Bansal-Yaron model does not have a closed-form solution for asset prices. The standard approach is **Campbell-Shiller log-linearisation** of the return identity:
$$
r_{w,t+1} \equiv \ln R_{w,t+1} \approx \kappa_0 + \kappa_1 z_{t+1} - z_t + g_{t+1},
$$
where $z_t = \ln(P_t/C_t)$ is the log consumption-wealth ratio (the variable to be solved for) and $\kappa_0$, $\kappa_1 \in (0,1)$ are linearisation constants.

**Guess**: The log price-consumption ratio is linear in the state variables:
$$
z_t = A_0 + A_1 x_t + A_2 \sigma^2_t.
$$
Substituting into the Euler equation for the wealth portfolio ($E_t[M_{t+1} R_{w,t+1}] = 1$, which in logs becomes $E_t[m_{t+1} + r_{w,t+1}] + \frac{1}{2}\text{Var}_t[m_{t+1} + r_{w,t+1}] = 0$), one can match coefficients to obtain:

$$
A_1 = \frac{1 - 1/\psi}{1 - \kappa_1 \rho}, \qquad
A_2 = \frac{(1-\gamma)(1 - 1/(2\theta))}{1 - \kappa_1 \nu_1} \cdot \frac{\phi_e^2/2 \cdot A_1^2 + \ldots}{\ldots}.
$$

**Key interpretation of $A_1$**:
- When $\psi > 1$: $1 - 1/\psi > 0$, so $A_1 > 0$. A positive shock to $x_t$ (higher expected future growth) *raises* the price-consumption ratio. The agent, anticipating better times, values the wealth claim more.
- When $\psi < 1$: $A_1 < 0$. The agent with low IES actually *lowers* the price-consumption ratio when growth improves — because they want to smooth consumption, they save less when times are expected to be good. This is the key reason the model requires $\psi > 1$: only then does good news about growth raise equity prices, making equity a bad hedge in bad times (negative $A_1$, bad times have low $x$, low $P/C$, low equity prices).

### 2.4 Sources of the Risk Premium

The log return on the equity claim (dividend claim) can similarly be log-linearised as:
$$
r^e_{t+1} \approx \kappa^e_0 + \kappa^e_1 z^e_{t+1} - z^e_t + g^d_{t+1},
$$
where $z^e_t = A^e_0 + A^e_1 x_t + A^e_2 \sigma^2_t$ is the log price-dividend ratio.

The equity risk premium decomposes into three terms:
$$
E_t[r^e_{t+1}] - r_{f,t} + \frac{1}{2}\text{Var}_t[r^e_{t+1}] =
\underbrace{\gamma \, \beta_{\eta}^e \, \sigma^2_t}_{\text{Short-run risk}}
+ \underbrace{(1-\gamma)\kappa_1 A_1 \phi_e \, \beta_{e,\sigma}^e \, \sigma^2_t}_{\text{Long-run risk}}
+ \underbrace{(\ldots) \sigma_w}_{\text{Vol-of-vol risk}},
$$
where $\beta^e_{\eta}$ and $\beta^e_{e}$ are the loadings of equity returns on the short-run ($\eta$) and long-run ($e$) consumption shocks.

Because the long-run shock has leverage $\phi_d$ and is amplified by the factor $(1-\kappa_1\rho)^{-1}$ (the persistence amplifier), even a small $\phi_e$ generates a large long-run risk premium when $\rho$ is close to one.

In [ ]:
# Solve the Bansal-Yaron model via log-linearisation
# Approximate A1, A2 for the wealth portfolio and the equity claim

# Log-linearisation constant kappa_1 depends on the mean log PD ratio
# We iterate to find a fixed point

def solve_by_model(gamma, psi, beta, rho, phi_e, phi_d, varphi,
                   mu_c, sigma_bar, nu1, sigma_w):
    """
    Solve Bansal-Yaron model via log-linearisation.
    Returns: (rf, ep, sigma_ep, kappa) dictionary of annualised moments.
    """
    theta = (1 - gamma) / (1 - 1/psi)

    # Iterate on kappa_1 (Campbell-Shiller linearisation constant)
    kappa1 = 0.997  # starting guess (monthly)
    for _ in range(200):
        # Solve for A1 (loading of log P/C on x_t)
        A1 = (1 - 1/psi) / (1 - kappa1 * rho)

        # Solve for A2 (loading of log P/C on sigma^2_t)
        # Numerator: variance terms from the Euler equation
        # sigma^2 coefficient in the SDF:
        # Var_t[m_{t+1}] / 2 + covariance terms
        # Simplified: A2 = theta * [0.5*(A1^2 * phi_e^2 + theta^{-1}*(1-gamma)^{-1}*...)] / (1 - kappa1*nu1)
        # Using BY (2004) Appendix expressions:
        numer_A2 = 0.5 * theta * (kappa1 * A1 * phi_e)**2 + 0.5 * (1 - gamma/psi)**2 - 0.5
        A2 = numer_A2 / (1 - kappa1 * nu1)

        # Update kappa_1 from the mean log PD ratio
        # Mean z = A0 + A2 * sigma_bar^2 (A1*E[x]=0)
        # kappa_0 + kappa_1*z_ss = r_w - g_c  (in steady state)
        # By definition: exp(z_ss) / (exp(z_ss)+1) = kappa_1 (Campbell-Shiller)
        # A0 is determined by the riskfree rate equation (not needed for kappa)
        # Approximate: use log(P/C) ≈ log(P/D) for calibration
        mean_z = np.log(20.0)  # approximate; use data PD ~20 monthly
        kappa1_new = np.exp(mean_z) / (1 + np.exp(mean_z))
        if abs(kappa1_new - kappa1) < 1e-8:
            break
        kappa1 = 0.5 * kappa1 + 0.5 * kappa1_new

    # Risk-free rate (monthly, conditional on sigma^2_t = sigma_bar^2)
    # r_f = -log(beta) + mu_c/psi - (1/2) * (gamma/psi)^2 * sigma^2
    #       - (1/2) * theta * (kappa1 * A1 * phi_e)^2 * sigma^2
    #       - A2 * (1 - nu1) * sigma^2
    sigma2_bar = sigma_bar**2
    rf_monthly = (-np.log(beta)
                  + mu_c / psi
                  - 0.5 * (gamma/psi)**2 * sigma2_bar
                  - 0.5 * theta * (kappa1 * A1 * phi_e)**2 * sigma2_bar
                  + A2 * sigma_w)  # vol-of-vol correction (small)

    # Equity: solve A1^e, A2^e for dividend claim
    # phi_d leverage on x; kappa1^e ≈ kappa1 (same linearisation)
    kappa1_e = kappa1
    A1_e = (phi_d - 1/psi) / (1 - kappa1_e * rho)
    numer_A2_e = (0.5 * theta * (kappa1 * A1 * phi_e)**2
                  + 0.5 * (varphi**2 - (gamma/psi)**2 + (kappa1_e * A1_e * phi_e)**2 * theta))
    A2_e = numer_A2_e / (1 - kappa1_e * nu1)

    # Equity premium (monthly)
    # EP = gamma*beta_eta*sigma^2 + (gamma - theta*kappa1*A1*phi_e)*(kappa1_e*A1_e*phi_e)*sigma^2
    # Simplified (BY Appendix C):
    beta_eta_e   = 1.0          # equity loading on short-run shock (normalised)
    ep_sr = gamma * sigma2_bar  # short-run contribution
    ep_lr = (gamma - theta * kappa1 * A1 * phi_e) * (kappa1_e * A1_e * phi_e) * sigma2_bar
    ep_monthly = ep_sr + ep_lr

    return {
        'rf':    rf_monthly * 12 * 100,
        'ep':    ep_monthly * 12 * 100,
        'A1':    A1,
        'A2':    A2,
        'kappa1': kappa1,
        'ep_sr_share': ep_sr / (ep_sr + ep_lr + 1e-12),
        'ep_lr_share': ep_lr / (ep_sr + ep_lr + 1e-12),
    }

# Baseline calibration
res_base = solve_by_model(gamma_by, psi_by, beta_by, rho, phi_e, phi_d, varphi,
                          mu_c, sigma_bar, nu1, sigma_w)

print("=" * 65)
print("Bansal-Yaron Model — Analytical Moments (annualised)")
print(f"  Parameters: γ={gamma_by}, ψ={psi_by}, β_monthly={beta_by}")
print("=" * 65)
print(f"  Riskfree rate rf:           {res_base['rf']:.2f}%   (data: ~0.9%)")
print(f"  Equity premium EP:          {res_base['ep']:.2f}%   (data: ~6.0%)")
print(f"  Short-run risk share:       {res_base['ep_sr_share']*100:.1f}%")
print(f"  Long-run risk share:        {res_base['ep_lr_share']*100:.1f}%")
print(f"  Log P/C loading on x (A1):  {res_base['A1']:.2f}")
print(f"  Log-linearisation κ₁:       {res_base['kappa1']:.4f}")

In [ ]:
# Sensitivity analysis: how do the model moments vary with psi and gamma?
psi_range   = np.linspace(0.5, 2.5, 30)
gamma_range = np.linspace(5, 25, 30)

rf_grid = np.full((len(gamma_range), len(psi_range)), np.nan)
ep_grid = np.full((len(gamma_range), len(psi_range)), np.nan)

for i, gam in enumerate(gamma_range):
    for j, psi in enumerate(psi_range):
        try:
            res = solve_by_model(gam, psi, beta_by, rho, phi_e, phi_d, varphi,
                                 mu_c, sigma_bar, nu1, sigma_w)
            if np.isfinite(res['rf']) and np.isfinite(res['ep']):
                rf_grid[i, j] = res['rf']
                ep_grid[i, j] = res['ep']
        except Exception:
            pass

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

PSI, GAM = np.meshgrid(psi_range, gamma_range)

ax = axes[0]
cf1 = ax.contourf(PSI, GAM, np.clip(ep_grid, 0, 15), levels=20, cmap='RdYlGn')
ax.contour(PSI, GAM, ep_grid, levels=[6.0], colors='white', linewidths=2)
plt.colorbar(cf1, ax=ax, label='Equity premium (% p.a.)')
ax.set_xlabel('IES $\\psi$')
ax.set_ylabel('Risk aversion $\\gamma$')
ax.set_title('Equity Premium (white = 6% data target)\nBansal-Yaron Model')
ax.scatter([psi_by], [gamma_by], color='white', s=100, marker='*',
           label=f'Baseline (ψ={psi_by}, γ={gamma_by})', zorder=5)
ax.legend(fontsize=9)
ax.axvline(1.0, color='white', lw=1, ls='--', alpha=0.5)
ax.text(0.85, 22, 'ψ<1', color='white', fontsize=9)
ax.text(1.05, 22, 'ψ>1', color='white', fontsize=9)

ax2 = axes[1]
cf2 = ax2.contourf(PSI, GAM, np.clip(rf_grid, -5, 10), levels=20, cmap='coolwarm')
ax2.contour(PSI, GAM, rf_grid, levels=[0.9], colors='black', linewidths=2)
plt.colorbar(cf2, ax=ax2, label='Riskfree rate (% p.a.)')
ax2.set_xlabel('IES $\\psi$')
ax2.set_ylabel('Risk aversion $\\gamma$')
ax2.set_title('Riskfree Rate (black = 0.9% data target)\nBansal-Yaron Model')
ax2.scatter([psi_by], [gamma_by], color='black', s=100, marker='*',
            label=f'Baseline (ψ={psi_by}, γ={gamma_by})', zorder=5)
ax2.legend(fontsize=9)
ax2.axvline(1.0, color='black', lw=1, ls='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("The BY model can match BOTH moments jointly: ψ>1 keeps rf low; γ>1 drives the EP.")
print("The IES threshold ψ=1 is critical: below it, the equity premium can become negative.")

### 2.5 Stochastic Volatility and the Price of Variance Risk

The second state variable $\sigma^2_t$ introduces **time-varying risk premia**. When $\sigma^2_t$ is high:
1. Equity is riskier — both short-run and long-run consumption shocks are amplified.
2. The equity risk premium rises.
3. The price-dividend ratio falls (via the $A_2 \sigma^2_t$ term, and $A_2 < 0$ under the calibration).

This generates **countercyclical risk premia**: premia are highest in recessions, when volatility is elevated. This is a key empirical feature of financial markets that the model successfully reproduces.

Furthermore, $\sigma^2_t$ is itself a risk factor. The **variance risk premium** — the difference between the risk-neutral and physical expectation of future variance — is:
$$
\text{VRP}_t = E^Q_t[\sigma^2_{t+1}] - E^P_t[\sigma^2_{t+1}] = -(1-\theta) \kappa_1 A_2 \sigma_w \cdot \sigma^2_t.
$$
Under the calibration, $\theta < 0$, $A_2 < 0$, so VRP $> 0$ — investors pay a premium to shed variance risk. This prediction aligns with the empirical finding (Bollerslev, Tauchen, Zhou, 2009) that the VRP predicts future excess returns with a positive sign.

---

# Part B — Disaster Risk

---

## 3. The Rietz-Barro Framework

The long-run risks model resolves the equity premium puzzle by introducing a *persistent* small component in consumption growth that is invisible in short samples. A fundamentally different approach is to introduce *rare large disasters* — events that are catastrophic but so infrequent that they barely appear in the historical US record.

### 3.1 The Core Argument (Rietz, 1988)

Rietz (1988) proposed that the equity premium reflects compensation for the small probability of a catastrophic event (a Great Depression-scale collapse) that would devastate consumption and equity payoffs simultaneously.

**Why this works intuitively**: An agent with CRRA utility $\gamma > 1$ has marginal utility that becomes *very* large when consumption is very low. A 50% drop in consumption raises marginal utility by $2^\gamma$. At $\gamma = 4$, this is a 16-fold increase. Even a 1% probability of such an event generates a large risk premium if the payoff in that state is sufficiently bad.

Rietz's proposal was criticised by Mehra and Prescott as requiring disaster probabilities that were too high to be realistic. Barro (2006) revived the idea using international historical data.

### 3.2 Barro (2006): The International Evidence

Barro (2006) compiled data on per-capita GDP contractions of 15% or more from 35 countries over the 20th century. He found:

- Probability of a disaster in any given year: $p \approx 0.017$ (about 1.7%).
- Given a disaster, the mean log contraction: $E[\ln(1-b)] \approx -0.22$ (about 22%).
- Disasters have significant variance: contractions range from 15% to over 60%.

Postwar US data covers only about 60 years — so we would expect to observe roughly $0.017 \times 60 \approx 1$ disaster on average. The US was lucky to avoid one, but the risk is priced in equity markets.

### 3.3 Formal Setup

**Consumption process**: Each period, with probability $1 - p$, a normal event occurs and consumption grows as $C_{t+1}/C_t = e^{\mu_c + \sigma_c \varepsilon_{t+1}}$. With probability $p$, a disaster occurs and consumption contracts by a random fraction $b_t \in (0,1)$:
$$
C_{t+1}/C_t = \begin{cases} e^{\mu_c + \sigma_c \varepsilon_{t+1}} & \text{with prob } 1-p \\ (1-b_t) e^{\mu_c + \sigma_c \varepsilon_{t+1}} & \text{with prob } p. \end{cases}
$$

**Equity payoff**: In a disaster, equities can be partially defaulted upon. Let the equity recovery rate be $f \leq 1$, so the dividend during a disaster falls by a factor $(1-b_t)^\phi$ for leverage $\phi \geq 1$.

### 3.4 The Equity Premium Under Disaster Risk

With CRRA utility (for clarity), the SDF is:
$$
M_{t+1} = \beta \left(\frac{C_{t+1}}{C_t}\right)^{-\gamma}.
$$
The riskfree rate and equity premium under i.i.d. Poisson disasters (using the Poisson approximation $p \ll 1$) are:
$$
r_f \approx -\ln\beta + \gamma \mu_c - \frac{\gamma^2 \sigma_c^2}{2} - p \cdot E[(1-b)^{-\gamma} - 1],
$$
$$
E[r^e] - r_f \approx \gamma \sigma_c^2 + p \cdot E[(1-b)^{-\gamma} (1 - (1-b)^{\phi})].
$$

The first term in the equity premium is the usual CCAPM term — small as before. The second term is the **disaster premium**: it depends on the probability $p$, the risk aversion $\gamma$, and the leverage $\phi$. Even with $\gamma = 3$ (very moderate), a 1.7% disaster probability with a 22% mean contraction can generate a 4–5% equity premium.

In [ ]:
# Compute equity premium and riskfree rate under disaster risk
# Barro (2006) calibration

# Normal-times parameters
mu_c_barro  = 0.025   # mean log consumption growth (excluding disaster)
sigma_c_barro = 0.02  # std of log consumption growth in normal times
beta_barro  = 0.95    # annual discount factor
p_barro     = 0.017   # annual disaster probability
phi_barro   = 3.0     # leverage of dividends

# Distribution of disaster size b (log-uniform approximation from Barro 2006)
# b is the fractional loss in consumption: C falls to (1-b)*C
# From Barro's data: b ranges from 0.15 to 0.64, mean ~0.29
N_disaster = 100_000
# Simulate b from a distribution that matches Barro's empirical moments
b_samples = rng.uniform(0.15, 0.64, N_disaster)

def disaster_moments(gamma, p, b_samples, mu_c, sigma_c, beta, phi):
    """Compute equity premium and riskfree rate under disaster risk (CRRA utility)."""
    b = b_samples
    one_minus_b = 1 - b

    # E[(1-b)^{-gamma}] — disaster contribution to risk-free rate (lowers it via precaution)
    E_M_disaster = np.mean(one_minus_b**(-gamma))

    # Riskfree rate
    # r_f = -log(beta) + gamma*mu_c - 0.5*gamma^2*sigma_c^2 - p*(E[(1-b)^{-gamma}] - 1)
    rf = (-np.log(beta)
          + gamma * mu_c
          - 0.5 * gamma**2 * sigma_c**2
          - p * (E_M_disaster - 1))

    # Equity premium
    # Normal-times: gamma*sigma_c^2 (same as CCAPM)
    ep_normal = gamma * sigma_c**2

    # Disaster term: p * E[(1-b)^{-gamma} * (1 - (1-b)^phi)]
    # = p * (E[(1-b)^{-gamma}] - E[(1-b)^{phi-gamma}])
    E_disaster_premium = np.mean(one_minus_b**(-gamma) * (1 - one_minus_b**phi))
    ep_disaster = p * E_disaster_premium

    ep_total = ep_normal + ep_disaster

    return rf * 100, ep_total * 100, ep_normal * 100, ep_disaster * 100

# Sweep over gamma
gammas_barro = np.linspace(1.0, 8.0, 100)
rf_barro = []
ep_barro = []
ep_normal_barro = []
ep_disaster_barro = []

for g in gammas_barro:
    r, e, en, ed = disaster_moments(g, p_barro, b_samples, mu_c_barro,
                                    sigma_c_barro, beta_barro, phi_barro)
    rf_barro.append(r)
    ep_barro.append(e)
    ep_normal_barro.append(en)
    ep_disaster_barro.append(ed)

rf_barro = np.array(rf_barro)
ep_barro = np.array(ep_barro)
ep_normal_barro = np.array(ep_normal_barro)
ep_disaster_barro = np.array(ep_disaster_barro)

# Compare with no-disaster case
ep_no_disaster = gammas_barro * sigma_c_barro**2 * 100
rf_no_disaster = (-np.log(beta_barro) + gammas_barro * mu_c_barro
                  - 0.5 * gammas_barro**2 * sigma_c_barro**2) * 100

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

ax = axes[0]
ax.plot(gammas_barro, ep_barro, 'steelblue', lw=2.5, label='With disaster risk')
ax.plot(gammas_barro, ep_no_disaster, 'grey', lw=2, ls='--', label='No disaster (CCAPM)')
ax.axhline(6.0, color='tomato', lw=1.5, ls=':', label='Data target: 6%')
ax.set_xlabel('Risk aversion $\\gamma$')
ax.set_ylabel('Equity premium (% p.a.)')
ax.set_title('Equity Premium\nDisaster vs No-Disaster')
ax.legend(fontsize=9)
ax.set_ylim(0, 15)

ax2 = axes[1]
ax2.plot(gammas_barro, rf_barro, 'steelblue', lw=2.5, label='With disaster risk')
ax2.plot(gammas_barro, rf_no_disaster, 'grey', lw=2, ls='--', label='No disaster')
ax2.axhline(0.9, color='tomato', lw=1.5, ls=':', label='Data target: 0.9%')
ax2.set_xlabel('Risk aversion $\\gamma$')
ax2.set_ylabel('Riskfree rate (% p.a.)')
ax2.set_title('Riskfree Rate\nDisaster Lowers rf via Precaution')
ax2.legend(fontsize=9)
ax2.set_ylim(-5, 15)

ax3 = axes[2]
ax3.stackplot(gammas_barro, ep_normal_barro, ep_disaster_barro,
              labels=['Normal-times EP', 'Disaster premium'],
              colors=['#2980b9', '#e74c3c'], alpha=0.85)
ax3.set_xlabel('Risk aversion $\\gamma$')
ax3.set_ylabel('Equity premium components (% p.a.)')
ax3.set_title('Decomposition of Equity Premium\nDisaster Dominates')
ax3.legend(fontsize=9)
ax3.set_ylim(0, 15)

plt.tight_layout()
plt.show()

# Find gamma that matches 6% EP
from scipy.optimize import brentq
def ep_minus_target(g):
    _, e, _, _ = disaster_moments(g, p_barro, b_samples, mu_c_barro,
                                  sigma_c_barro, beta_barro, phi_barro)
    return e - 6.0

g_star = brentq(ep_minus_target, 1.0, 8.0)
rf_star, _, _, _ = disaster_moments(g_star, p_barro, b_samples, mu_c_barro,
                                    sigma_c_barro, beta_barro, phi_barro)
print(f"γ required to match 6% equity premium: {g_star:.2f}")
print(f"Implied riskfree rate at γ={g_star:.2f}: {rf_star:.2f}%  (data: ~0.9%)")
print(f"\nCompare: CCAPM requires γ ≈ {6.0 / (cov_g_r * 100):.0f} to match the same target.")

# Compute what fraction is disaster premium at gamma=g_star
_, _, en_star, ed_star = disaster_moments(g_star, p_barro, b_samples, mu_c_barro,
                                          sigma_c_barro, beta_barro, phi_barro)
cov_g_r = 0.20 * sigma_c_barro * 0.165  # approximate from Week 2
print(f"Disaster premium share: {ed_star/6.0*100:.0f}% of total equity premium")

### 3.5 The Peso Problem and Identification

The disaster risk model faces a fundamental empirical challenge: **disasters are rare by assumption**, so the postwar US data contains very few (arguably zero) clear consumption disasters. This raises the **peso problem**: how can we identify the probability and severity of events that have not occurred in our sample?

Barro's answer is to use international cross-country data, where disasters (wars, financial crises, hyperinflations) are more frequent. But this introduces a different issue: the consumption and equity market data of other countries may not be comparable to the US.

**Options market evidence**: If disaster risk is priced, options markets should reflect it. The implied volatility smile — the fact that out-of-the-money put options are more expensive than the Black-Scholes model predicts — can be interpreted as the market pricing the possibility of a large, sudden drop. Gabaix (2012) and others have formalised this connection.

**Identification strategy**: Nakamura et al. (2013) use the long-run international data (35 countries, 1890–2006) and a Bayesian approach to estimate disaster parameters jointly with the preference parameters. They find $p \approx 0.02$ and $E[b] \approx 0.30$, consistent with Barro (2006).

### 3.6 Long-Run Risks vs Disaster Risk: A Comparison

Both models resolve the equity premium puzzle. Their key differences are:

| Dimension | Long-Run Risks | Disaster Risk |
|---|---|---|
| **Mechanism** | Persistent small shocks priced by EZ agents | Rare large shocks priced by CRRA agents |
| **Preferences needed** | EZ with $\psi > 1$, $\gamma \approx 10$ | CRRA with $\gamma \approx 3$–5 |
| **Empirical basis** | Predictability, vol clustering | International disaster data |
| **Time-varying risk** | Yes (stochastic volatility) | Yes (time-varying $p$) |
| **Options pricing** | Stochastic vol channel | Jump-risk channel |
| **Criticism** | $\psi > 1$ debated; long-run component barely visible | Peso problem; disasters may not recur |

The field has not reached consensus. Both models are taken seriously in the literature, and ongoing work attempts to distinguish between them using options data, bond markets, and international cross-sections.

In [ ]:
# Simulate and compare consumption paths: LRR vs Disaster processes
T_compare = 60   # 60 years — the postwar US sample
n_sims    = 5000

# LRR: annual g_c = mu_c + x_t + sigma_c * eps
# Disaster: annual g_c = mu_c + sigma_c*eps with p=1.7% of a disaster b

# Annual LRR consumption levels (set sigma_c same as Barro for fair comparison)
mu_c_lrr   = 0.018
sigma_c_lrr = 0.027
rho_lrr    = 0.979
phi_e_lrr  = 0.038

# Draw LRR consumption paths
x_lrr = np.zeros((T_compare, n_sims))
g_lrr = np.zeros((T_compare, n_sims))
x_lrr[0] = 0
for t in range(T_compare - 1):
    eps_t = rng.standard_normal(n_sims)
    e_t   = rng.standard_normal(n_sims)
    g_lrr[t]    = mu_c_lrr + x_lrr[t] + sigma_c_lrr * eps_t
    x_lrr[t+1] = rho_lrr * x_lrr[t] + phi_e_lrr * sigma_c_lrr * e_t
g_lrr[-1] = mu_c_lrr + x_lrr[-1] + sigma_c_lrr * rng.standard_normal(n_sims)

# Cumulative consumption (log level, starting at 0)
log_C_lrr = np.cumsum(g_lrr, axis=0)

# Draw Disaster consumption paths
g_disaster = rng.normal(mu_c_barro, sigma_c_barro, (T_compare, n_sims))
disaster_indicator = rng.random((T_compare, n_sims)) < p_barro
b_draw = rng.uniform(0.15, 0.64, (T_compare, n_sims))
g_disaster += disaster_indicator * np.log(1 - b_draw)

log_C_disaster = np.cumsum(g_disaster, axis=0)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

years_c = np.arange(T_compare)

# Panel 1: sample paths, 10 draws each
ax = axes[0]
for j in range(10):
    ax.plot(years_c, log_C_lrr[:, j] * 100, color='steelblue', lw=0.8, alpha=0.6)
for j in range(10):
    ax.plot(years_c, log_C_disaster[:, j] * 100, color='tomato', lw=0.8, alpha=0.6,
            ls='--')
ax.plot([], [], 'steelblue', lw=1.5, label='Long-run risks')
ax.plot([], [], 'tomato', lw=1.5, ls='--', label='Disaster risk')
ax.set_xlabel('Year'); ax.set_ylabel('Log consumption (pp)')
ax.set_title('Sample Paths (10 draws, 60 years)\nDisaster vs Long-Run Risks')
ax.legend()

# Panel 2: distribution of total 60-year log consumption change
ax2 = axes[1]
ax2.hist(log_C_lrr[-1] * 100, bins=60, density=True, alpha=0.6,
         color='steelblue', label='LRR')
ax2.hist(log_C_disaster[-1] * 100, bins=60, density=True, alpha=0.6,
         color='tomato', label='Disaster')
ax2.set_xlabel('60-year cumulative log consumption change (pp)')
ax2.set_ylabel('Density')
ax2.set_title('Distribution of Long-Run Outcomes\nDisaster has heavier left tail')
ax2.legend()

# Panel 3: autocorrelation of consumption growth
ax3 = axes[2]
lags = np.arange(1, 11)
ac_lrr      = [np.corrcoef(g_lrr[:-k].flatten(), g_lrr[k:].flatten())[0,1] for k in lags]
ac_disaster = [np.corrcoef(g_disaster[:-k].flatten(), g_disaster[k:].flatten())[0,1] for k in lags]
ax3.bar(lags - 0.2, ac_lrr, 0.35, label='LRR', color='steelblue', alpha=0.85)
ax3.bar(lags + 0.2, ac_disaster, 0.35, label='Disaster', color='tomato', alpha=0.85)
ax3.axhline(0, color='black', lw=0.8)
ax3.set_xlabel('Lag (years)'); ax3.set_ylabel('Autocorrelation')
ax3.set_title('Autocorrelation of Consumption Growth\nLRR has persistent structure')
ax3.legend()

plt.tight_layout()
plt.show()

print("Key distinction: the LRR model generates persistent autocorrelation in consumption growth.")
print("The disaster model generates a heavier left tail but little autocorrelation.")
print("These different predictions can (in principle) be tested empirically.")

---

# Part C — The Arbitrage Pricing Theory and Factor Models

---

## 4. From Equilibrium to Empirical Factors

Parts A and B derived specific equilibrium models of the SDF. These models make strong structural predictions — both about the level of expected returns (equity premium, riskfree rate) and about the cross-sectional variation (which assets have high expected returns and why).

However, the equilibrium models require strong assumptions about preferences, the consumption process, and market structure. An alternative, more agnostic approach is to characterise the SDF empirically — to ask: what observable factors explain the cross-sectional variation in expected returns, without requiring a full structural model?

This is the empirical asset pricing programme, which begins with the Arbitrage Pricing Theory.

---

## 5. The Arbitrage Pricing Theory (Ross, 1976)

### 5.1 The Factor Structure

**Assumption**: Returns are generated by a $K$-factor model:
$$
R^i = E[R^i] + \sum_{k=1}^{K} \beta_{ik} f_k + \varepsilon^i,
$$
where:
- $f_k$ are zero-mean common factors, $\text{Cov}(f_k, \varepsilon^i) = 0$ for all $i, k$.
- $\varepsilon^i$ is idiosyncratic noise: $\text{Cov}(\varepsilon^i, \varepsilon^j) = 0$ for $i \neq j$ (approximate factor structure).
- $\beta_{ik}$ is asset $i$'s loading ("beta") on factor $k$.

This is a purely statistical assumption about the return covariance structure. No utility functions, no representative agents.

### 5.2 The APT Pricing Result

**Theorem (Ross, 1976)**: In an economy with a factor structure as above, if there are no approximate arbitrage opportunities (formal definition below), then there exist constants $\lambda_0, \lambda_1, \ldots, \lambda_K$ such that:
$$
\boxed{E[R^i] = \lambda_0 + \sum_{k=1}^{K} \beta_{ik} \lambda_k,}
$$
where $\lambda_0 = R_f$ (the riskfree rate) and $\lambda_k$ is the **factor risk premium** for factor $k$.

**Proof sketch (law of one price argument)**:
Consider a portfolio $w$ of $N$ assets that is:
1. Zero-cost: $\mathbf{w} \cdot \mathbf{1} = 0$.
2. Zero-beta on all factors: $\mathbf{w} \cdot \boldsymbol{\beta}_k = 0$ for all $k$.
3. Well-diversified: the idiosyncratic component $\mathbf{w} \cdot \boldsymbol{\varepsilon} \to 0$ as $N \to \infty$.

Such a portfolio has zero risk (as $N \to \infty$) but may have non-zero expected return $\mathbf{w} \cdot E[\mathbf{R}]$. No-arbitrage requires this expected return to be zero. By the duality of linear programming, this implies that $E[\mathbf{R}]$ lies in the column space of $[\mathbf{1}, \boldsymbol{\beta}_1, \ldots, \boldsymbol{\beta}_K]$ — which is exactly the APT equation.

### 5.3 APT vs CAPM

The APT is more general than the CAPM in one sense and less precise in another:

- **More general**: The APT allows multiple factors and does not require a representative agent or market portfolio identification.
- **Less precise**: The APT does not identify the factors. It says expected returns are linear in betas, but does not say what the betas are. The CAPM identifies the factor as the market portfolio.
- **APT is approximate**: The no-arbitrage argument requires idiosyncratic risks to vanish — it works for well-diversified portfolios but may have large deviations for individual stocks.

**Connection to the SDF**: The APT factor structure implies that the SDF can be written as a linear function of the factors:
$$
M = a_0 + \sum_{k=1}^K a_k f_k.
$$
Each factor that appears in $M$ carries a risk premium. The APT is thus a model of $M$ — specifically, a multi-factor linear model.

In [ ]:
# Illustrate the APT: show that diversification eliminates idiosyncratic risk
# but beta risk remains even in large portfolios

N_assets_apt = 200   # total assets
K_factors    = 3     # number of factors
T_apt        = 120   # months (10 years)

# Factor risk premia
lambda_apt = np.array([0.005, 0.003, 0.004])  # monthly
Rf_apt = 0.001

# Factor covariance matrix (diagonal for simplicity)
factor_vols = np.array([0.045, 0.035, 0.030])  # monthly factor vols
Sigma_f = np.diag(factor_vols**2)

# Draw betas from uniform distribution
betas_apt = rng.uniform(0.2, 1.8, (N_assets_apt, K_factors))

# True expected excess returns
E_excess_apt = betas_apt @ lambda_apt

# Simulate factor realisations
f_sim = rng.normal(0, factor_vols, (T_apt, K_factors))

# Idiosyncratic noise (all assets, each period)
sigma_eps_apt = rng.uniform(0.06, 0.15, N_assets_apt)  # idiosyncratic vol (monthly)
eps_sim = rng.normal(0, 1, (T_apt, N_assets_apt)) * sigma_eps_apt[None, :]

# Realised returns
R_sim = (E_excess_apt[None, :]   # expected excess return
         + f_sim @ betas_apt.T   # factor component
         + eps_sim)              # idiosyncratic

# Build equal-weighted portfolios of size n and compute vol decomposition
portfolio_sizes = [1, 2, 5, 10, 20, 50, 100, 200]
systematic_var  = []
idiosyncratic_var = []
total_var       = []

for n in portfolio_sizes:
    # Take first n assets, equal weight
    w = np.ones(n) / n
    port_ret = R_sim[:, :n] @ w  # (T,)

    # Systematic component: factor exposure of portfolio
    beta_port = betas_apt[:n].T @ w  # (K,)
    sys_var = beta_port @ Sigma_f @ beta_port

    # Idiosyncratic component: shrinks with 1/n
    idio_var = np.sum(w**2 * sigma_eps_apt[:n]**2)

    systematic_var.append(sys_var)
    idiosyncratic_var.append(idio_var)
    total_var.append(port_ret.var())

systematic_var   = np.array(systematic_var)
idiosyncratic_var = np.array(idiosyncratic_var)
total_var        = np.array(total_var)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

ax = axes[0]
ax.plot(portfolio_sizes, systematic_var * 100**2, 'steelblue', lw=2, marker='o', ms=6,
        label='Systematic variance')
ax.plot(portfolio_sizes, idiosyncratic_var * 100**2, 'tomato', lw=2, marker='s', ms=6,
        label='Idiosyncratic variance')
ax.plot(portfolio_sizes, total_var * 100**2, 'k--', lw=1.5, marker='^', ms=5,
        label='Total portfolio variance')
ax.set_xscale('log')
ax.set_xlabel('Number of assets in portfolio')
ax.set_ylabel('Variance (×10⁴, monthly)')
ax.set_title('Variance Decomposition as Portfolio Size Grows\nIdiosyncratic risk diversifies away; systematic does not')
ax.legend()

# Panel 2: APT cross-sectional regression
# True vs estimated expected returns using known betas
# Fama-MacBeth: estimate betas in first half, price factors in second half
T_half = T_apt // 2

# Estimated betas (OLS in first half)
X_ts_apt = np.column_stack([np.ones(T_half), f_sim[:T_half]])
betas_estimated_apt = np.empty((N_assets_apt, K_factors))
for i in range(N_assets_apt):
    coef = np.linalg.lstsq(X_ts_apt, R_sim[:T_half, i], rcond=None)[0]
    betas_estimated_apt[i] = coef[1:]

# Cross-sectional regression in second half
E_R_second = R_sim[T_half:].mean(axis=0)  # realised mean returns
X_cs_apt = np.column_stack([np.ones(N_assets_apt), betas_estimated_apt])
lambda_hat = np.linalg.lstsq(X_cs_apt, E_R_second, rcond=None)[0]

E_R_predicted = X_cs_apt @ lambda_hat

ax2 = axes[1]
ax2.scatter(E_R_predicted * 1200, E_R_second * 1200,
            alpha=0.5, s=20, color='steelblue', edgecolors='none')
lims = [min(E_R_predicted.min(), E_R_second.min()) * 1200 - 0.5,
        max(E_R_predicted.max(), E_R_second.max()) * 1200 + 0.5]
ax2.plot(lims, lims, 'k--', lw=1.5, label='45-degree line')
ax2.set_xlabel('APT-predicted $E[R^i]$ (% p.a.)')
ax2.set_ylabel('Realised mean return (% p.a.)')
ax2.set_title(f'APT Cross-Sectional Fit\n($K={K_factors}$ factors, $N={N_assets_apt}$ assets)')
ax2.legend()

# R-squared
ss_res = np.sum((E_R_second - E_R_predicted)**2)
ss_tot = np.sum((E_R_second - E_R_second.mean())**2)
r2 = 1 - ss_res / ss_tot
ax2.text(0.05, 0.92, f'$R^2 = {r2:.2f}$', transform=ax2.transAxes, fontsize=11)

plt.tight_layout()
plt.show()

print(f"Estimated factor risk premia (annualised, %):\n"
      f"  λ₀={lambda_hat[0]*1200:.2f}  λ₁={lambda_hat[1]*1200:.2f}  "
      f"λ₂={lambda_hat[2]*1200:.2f}  λ₃={lambda_hat[3]*1200:.2f}")
print(f"True factor risk premia (annualised, %):      "
      f"  λ₀={Rf_apt*1200:.2f}  λ₁={lambda_apt[0]*1200:.2f}  "
      f"λ₂={lambda_apt[1]*1200:.2f}  λ₃={lambda_apt[2]*1200:.2f}")

---

## 6. The Fama-French Factor Models

### 6.1 From Theory to Empirical Factors

The APT tells us that expected returns are linear in factor betas. It does not tell us what the factors are. The empirical programme of Fama and French identifies factors by sorting stocks on observable firm characteristics and constructing zero-cost long-short portfolios.

**Key idea**: If a characteristic predicts expected returns in the cross-section — even after controlling for market beta — then it reveals information about an omitted factor in the SDF. The characteristic-sorted portfolio is a direct estimate of the priced factor.

### 6.2 The Fama-French Three-Factor Model (1993)

Fama and French (1993) documented that the CAPM leaves large, systematic residuals in the cross-section. In particular:
- **Small stocks** earn higher returns than large stocks, after controlling for beta.
- **Value stocks** (high book-to-market ratio, $B/M$) earn higher returns than growth stocks (low $B/M$), after controlling for beta.

They proposed two additional factors to capture these premia:

- **SMB** (Small Minus Big): The return on a portfolio long small-cap stocks and short large-cap stocks. Compensates for the size premium.
- **HML** (High Minus Low): The return on a portfolio long high-$B/M$ (value) stocks and short low-$B/M$ (growth) stocks. Compensates for the value premium.

The **Fama-French three-factor model**:
$$
\boxed{E[R^i] - R_f = \beta^{\text{MKT}}_i \lambda_{\text{MKT}} + \beta^{\text{SMB}}_i \lambda_{\text{SMB}} + \beta^{\text{HML}}_i \lambda_{\text{HML}}.}
$$

### 6.3 Construction of SMB and HML

**SMB** is constructed by:
1. At the end of June each year, rank all NYSE stocks by market capitalisation and split at the median.
2. Independently rank all NYSE stocks by $B/M$ and split into three groups (bottom 30%, middle 40%, top 30%).
3. Form six intersecting portfolios: SG (small growth), SN (small neutral), SV (small value), BG, BN, BV.
4. $\text{SMB} = \frac{1}{3}(\text{SG} + \text{SN} + \text{SV}) - \frac{1}{3}(\text{BG} + \text{BN} + \text{BV})$.

**HML** is constructed by:
$$
\text{HML} = \frac{1}{2}(\text{SV} + \text{BV}) - \frac{1}{2}(\text{SG} + \text{BG}).
$$

The construction ensures that each factor is independent of the other and of the market. Portfolios are rebalanced annually.

### 6.4 The Fama-French Five-Factor Model (2015)

Fama and French (2015) augmented the three-factor model with two additional factors motivated by the dividend discount model. If the price equals the present value of future dividends, then controlling for price (i.e., $B/M$), stocks with higher expected profitability or lower expected investment should have higher expected returns.

- **RMW** (Robust Minus Weak): Long firms with high operating profitability, short firms with low profitability.
- **CMA** (Conservative Minus Aggressive): Long firms with low asset growth (conservative investment), short firms with high asset growth.

The **five-factor model**:
$$
E[R^i] - R_f = \beta^{\text{MKT}}_i \lambda_{\text{MKT}} + \beta^{\text{SMB}}_i \lambda_{\text{SMB}} + \beta^{\text{HML}}_i \lambda_{\text{HML}} + \beta^{\text{RMW}}_i \lambda_{\text{RMW}} + \beta^{\text{CMA}}_i \lambda_{\text{CMA}}.
$$

**Important caveat**: Adding factors reduces the pricing errors by construction (more parameters), but raises the question of whether the new factors are genuinely priced risk factors or spurious overfitting. We return to this in the section on model evaluation.

In [ ]:
# Simulate Fama-French factor construction and time-series properties
# Use simulated data with known factor structure

# We simulate monthly factor returns calibrated to FF historical moments
# US data (1963-2023 approximate): annualised means and vols

# Factor means (annual %) and vols
factor_means_ann = np.array([8.5,  2.8,  3.8,  3.3, 2.3])   # MKT, SMB, HML, RMW, CMA
factor_vols_ann  = np.array([15.5, 10.5, 11.2, 8.0, 6.5])   # annualised vols
factor_names     = ['MKT', 'SMB', 'HML', 'RMW', 'CMA']

# Correlation matrix (approximate from data)
corr_ff = np.array([
    [ 1.00,  0.27, -0.37, -0.26, -0.39],
    [ 0.27,  1.00, -0.13, -0.38, -0.13],
    [-0.37, -0.13,  1.00,  0.10,  0.68],
    [-0.26, -0.38,  0.10,  1.00,  0.07],
    [-0.39, -0.13,  0.68,  0.07,  1.00],
])

# Monthly parameters
factor_means_m = factor_means_ann / 100 / 12
factor_vols_m  = factor_vols_ann  / 100 / np.sqrt(12)

Sigma_ff = np.outer(factor_vols_m, factor_vols_m) * corr_ff

T_ff = 12 * 60   # 60 years monthly

# Draw factor returns
L = cholesky(Sigma_ff, lower=True)
z = rng.standard_normal((T_ff, 5))
F_ff = z @ L.T + factor_means_m[None, :]  # (T, 5) factor returns

# Cumulative factor returns
cum_ff = np.cumsum(F_ff, axis=0)
years_ff = np.arange(T_ff) / 12

fig, axes = plt.subplots(2, 3, figsize=(15, 9))

colors_ff = ['#2c3e50', '#e74c3c', '#e67e22', '#27ae60', '#8e44ad']

for i, (name, color) in enumerate(zip(factor_names, colors_ff)):
    ax = axes[i // 3, i % 3]
    ax.plot(years_ff, cum_ff[:, i] * 100, color=color, lw=1.2)
    ax.axhline(0, color='black', lw=0.6, ls='--')
    mean_ann = F_ff[:, i].mean() * 1200
    vol_ann  = F_ff[:, i].std() * 100 * np.sqrt(12)
    sr       = mean_ann / vol_ann
    ax.set_xlabel('Year')
    ax.set_ylabel('Cumulative return (pp)')
    ax.set_title(f'{name} Factor\n$E[R]$={mean_ann:.1f}%, $\\sigma$={vol_ann:.1f}%, SR={sr:.2f}')

# Last panel: correlation heatmap
ax_corr = axes[1, 2]
corr_sim = np.corrcoef(F_ff.T)
im = ax_corr.imshow(corr_sim, cmap='RdBu_r', vmin=-1, vmax=1)
ax_corr.set_xticks(range(5)); ax_corr.set_yticks(range(5))
ax_corr.set_xticklabels(factor_names); ax_corr.set_yticklabels(factor_names)
plt.colorbar(im, ax=ax_corr)
for ii in range(5):
    for jj in range(5):
        ax_corr.text(jj, ii, f'{corr_sim[ii, jj]:.2f}', ha='center', va='center',
                     fontsize=9, color='black' if abs(corr_sim[ii, jj]) < 0.5 else 'white')
ax_corr.set_title('Simulated Factor Correlations')

plt.tight_layout()
plt.suptitle('Fama-French Five Factors: Simulated Time Series', fontsize=13,
             fontweight='bold', y=1.01)
plt.show()

print("Note: HML and CMA are highly correlated (~0.68 in data).")
print("This raises the question of whether both are separately necessary.")
print("Indeed, HML becomes redundant in FF5 for many test portfolios.")

---

## 7. Evaluating Factor Models: The GRS Test and Pricing Errors

### 7.1 Time-Series Regressions and Alpha

The standard test of a factor model is a time-series regression of each test asset's excess return on the proposed factors:
$$
R^i_t - R_{f,t} = \alpha_i + \sum_{k=1}^{K} \beta_{ik} F_{k,t} + \varepsilon_{i,t}.
$$
If the model is correctly specified, **all $\alpha_i$ should be zero**. A non-zero $\alpha_i$ ("alpha" or "pricing error") means the model fails to explain the average return of asset $i$.

**Statistical test (Gibbons-Ross-Shanken, 1989)**: The GRS statistic tests $H_0: \alpha_1 = \alpha_2 = \ldots = \alpha_N = 0$ jointly:
$$
\text{GRS} = \frac{T(T - N - K)}{N(T - K - 1)} \cdot \hat{\boldsymbol{\alpha}}' \hat{\boldsymbol{\Sigma}}^{-1} \hat{\boldsymbol{\alpha}} \cdot \left(1 + \bar{\mathbf{f}}' \hat{\boldsymbol{\Omega}}^{-1} \bar{\mathbf{f}}\right)^{-1} \sim F_{N, T-N-K},
$$
where:
- $\hat{\boldsymbol{\alpha}}$ is the vector of estimated intercepts.
- $\hat{\boldsymbol{\Sigma}}$ is the $N \times N$ covariance matrix of residuals.
- $\bar{\mathbf{f}}$ is the vector of sample mean factor returns.
- $\hat{\boldsymbol{\Omega}}$ is the $K \times K$ factor covariance matrix.

The GRS statistic has a geometric interpretation: it measures the improvement in the maximum Sharpe ratio achievable by adding the test assets to the factor portfolio. If all alphas are zero, adding the test assets does not improve the maximum Sharpe ratio.

### 7.2 The Sharpe Ratio Interpretation

Let $SR_f$ be the Sharpe ratio of the tangency portfolio formed from the $K$ factors alone, and $SR_{f+}$ be the Sharpe ratio of the tangency portfolio formed from the factors plus all $N$ test assets. Then:
$$
SR_{f+}^2 = SR_f^2 + \frac{\hat{\boldsymbol{\alpha}}' \hat{\boldsymbol{\Sigma}}^{-1} \hat{\boldsymbol{\alpha}}}{1}.
$$
The GRS statistic is proportional to $SR_{f+}^2 - SR_f^2$. A factor model passes the GRS test when including the test assets does not improve the Sharpe ratio — meaning the factors already span the efficient frontier.

### 7.3 The HJ Distance

An alternative to the GRS test is the **Hansen-Jagannathan (HJ) distance**, introduced in Week 1. For a candidate SDF $M^\theta$ parameterised by $\theta$, the HJ distance is:
$$
\delta(\theta) = \min_{\theta} \sqrt{E\left[(M^\theta - M^*)^2\right]},
$$
where $M^*$ is the minimum-variance SDF that prices the test assets exactly. The HJ distance can be estimated as:
$$
\hat{\delta}^2(\theta) = (\mathbf{1} - E[M^\theta \mathbf{R}])' \left(E[\mathbf{R}\mathbf{R}']\right)^{-1} (\mathbf{1} - E[M^\theta \mathbf{R}]).
$$
It is interpretable in the same units as the SDF (ratio of marginal utilities), making it easier to compare across models.

In [ ]:
# Implement the GRS test and compare CAPM vs FF3 vs FF5
# Use simulated 25 size-value sorted portfolios with a known 5-factor structure

T_grs = 600   # 50 years monthly
N_grs = 25    # 25 test portfolios (5x5 size-value sorts)

# Simulate: true model is FF5 (all 5 factors matter)
# CAPM will have large alphas; FF3 smaller; FF5 near zero

# Draw factor returns (same as above, but fresh draw)
z2 = rng.standard_normal((T_grs, 5))
F_grs = z2 @ L.T + factor_means_m[None, :]   # (T_grs, 5)

# Assign betas to 25 portfolios (structured by size and value)
# Size: rows (1=small, 5=large); Value: cols (1=growth, 5=value)
size_idx  = np.repeat(np.arange(5), 5)   # 0..4
value_idx = np.tile(np.arange(5), 5)     # 0..4

# MKT beta close to 1 for all; SMB decreases with size; HML increases with value
beta_MKT = 0.9 + 0.05 * rng.standard_normal(N_grs)
beta_SMB = 1.0 - 0.25 * size_idx  + 0.05 * rng.standard_normal(N_grs)
beta_HML = -0.4 + 0.20 * value_idx + 0.05 * rng.standard_normal(N_grs)
beta_RMW = 0.1 + 0.05 * rng.standard_normal(N_grs)   # small RMW exposure
beta_CMA = 0.05 + 0.04 * rng.standard_normal(N_grs)  # small CMA exposure

betas_grs = np.column_stack([beta_MKT, beta_SMB, beta_HML, beta_RMW, beta_CMA])

# True expected excess returns from FF5 model
E_excess_grs = betas_grs @ factor_means_m  # (N_grs,)

# Simulate portfolio excess returns
sigma_eps_grs = 0.02  # monthly idiosyncratic vol (well-diversified portfolios)
eps_grs = rng.normal(0, sigma_eps_grs, (T_grs, N_grs))
R_grs = E_excess_grs[None, :] + F_grs @ betas_grs.T + eps_grs  # (T, N)

def run_grs_test(R_test, F_factors):
    """Run GRS test of H0: all alphas = 0 in time-series regression on F_factors."""
    T, N = R_test.shape
    K = F_factors.shape[1]

    # Add intercept to factor matrix
    X = np.column_stack([np.ones(T), F_factors])

    # OLS regression for each portfolio
    coefs = np.linalg.lstsq(X, R_test, rcond=None)[0]  # (1+K, N)
    alphas = coefs[0]     # (N,)
    resids = R_test - X @ coefs  # (T, N)

    # Residual covariance matrix
    Sigma_hat = resids.T @ resids / (T - K - 1)  # (N, N)

    # Factor moments
    f_bar = F_factors.mean(axis=0)   # (K,)
    Omega_hat = np.cov(F_factors.T)  # (K, K)

    # GRS statistic
    Sigma_inv = np.linalg.inv(Sigma_hat)
    if K == 1:
        Omega_inv_scalar = 1.0 / Omega_hat
        scaling = 1 + f_bar[0] * Omega_inv_scalar * f_bar[0]
    else:
        scaling = 1 + f_bar @ np.linalg.inv(Omega_hat) @ f_bar

    grs_stat = (T * (T - N - K)) / (N * (T - K - 1)) * (alphas @ Sigma_inv @ alphas) / scaling

    # Degrees of freedom
    from scipy.stats import f as f_dist
    p_value = 1 - f_dist.cdf(grs_stat, N, T - N - K)

    # Mean absolute alpha and cross-sectional R2
    mean_abs_alpha = np.abs(alphas).mean() * 1200  # annualised %

    return {
        'alphas': alphas,
        'grs_stat': grs_stat,
        'p_value': p_value,
        'mean_abs_alpha': mean_abs_alpha,
        'Sigma': Sigma_hat,
    }

# Test three models
res_capm = run_grs_test(R_grs, F_grs[:, [0]])        # CAPM: MKT only
res_ff3  = run_grs_test(R_grs, F_grs[:, [0,1,2]])    # FF3: MKT, SMB, HML
res_ff5  = run_grs_test(R_grs, F_grs)                 # FF5: all five

print("=" * 65)
print(f"GRS Test Results — {N_grs} portfolios, {T_grs} months")
print("=" * 65)
for name, res in [('CAPM', res_capm), ('FF3', res_ff3), ('FF5', res_ff5)]:
    print(f"\n{name}:")
    print(f"  GRS statistic:        {res['grs_stat']:.3f}")
    print(f"  p-value:              {res['p_value']:.4f}")
    print(f"  Mean |alpha| (% p.a.): {res['mean_abs_alpha']:.3f}")
    print(f"  Reject H0 (α=5%)?     {'YES' if res['p_value'] < 0.05 else 'NO'}")

# Visualise alphas
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
labels_25 = [f'({s+1},{v+1})' for s in range(5) for v in range(5)]
x25 = np.arange(N_grs)

for ax, (name, res) in zip(axes, [('CAPM', res_capm), ('FF3', res_ff3), ('FF5', res_ff5)]):
    alphas_ann = res['alphas'] * 1200
    se_alpha   = np.sqrt(np.diag(res['Sigma'])) / np.sqrt(T_grs) * 1200
    colors_alpha = ['#e74c3c' if a > 0 else '#2980b9' for a in alphas_ann]
    ax.bar(x25, alphas_ann, color=colors_alpha, alpha=0.75, edgecolor='black', lw=0.3)
    ax.errorbar(x25, alphas_ann, yerr=2*se_alpha, fmt='none', color='black', capsize=2, lw=0.8)
    ax.axhline(0, color='black', lw=1)
    ax.set_xlabel('Portfolio (size, value)')
    ax.set_ylabel('Alpha (% p.a.)')
    ax.set_title(f'{name}\nGRS={res["grs_stat"]:.2f}, p={res["p_value"]:.3f}, '
                 f'|ᾱ|={res["mean_abs_alpha"]:.2f}%')
    ax.set_xticks(x25[::5])
    ax.set_xticklabels(labels_25[::5], fontsize=8)

plt.tight_layout()
plt.suptitle('Pricing Errors (Alphas) for 25 Size-Value Portfolios\n'
             'Red = positive alpha, Blue = negative alpha; error bars = ±2 s.e.',
             fontsize=12, y=1.02)
plt.show()

---

## 8. The Factor Zoo and the Multiple Testing Problem

### 8.1 The Proliferation of Factors

The empirical asset pricing literature has produced a vast number of proposed factors since Fama and French (1993). Harvey, Liu, and Zhu (2016) documented over 316 factors published in leading journals by 2014 — a number that has grown substantially since. These factors include momentum, quality, low volatility, accruals, asset growth, earnings surprises, and dozens more.

Cochrane (2011) referred to this as the **factor zoo**: a proliferation of empirical factors with little theoretical foundation, many of which may be the result of data mining rather than genuine risk premia.

### 8.2 The Multiple Testing Problem

The standard criterion for declaring a new factor significant is a $t$-statistic above 2.0 (corresponding to a $p$-value below 5%). But if 316 factors are tested independently, the expected number of false positives under the null of no true factor is:
$$
E[\text{false positives}] = 316 \times 0.05 = 15.8.
$$
This is the **multiple testing problem**: the more factors we test, the more likely we are to find spurious ones by chance.

**Harvey, Liu, and Zhu (2016) proposal**: The threshold for claiming a new factor is significant should account for the number of tests conducted. Using a Bayesian framework, they argue that the appropriate threshold for the $t$-statistic of a new factor should be approximately 3.0 (rather than 2.0) to maintain a false discovery rate below 5%.

**Implications**: Many published factors with $t$-statistics between 2.0 and 3.0 may be false discoveries. The expected number of true factors is far smaller than 316.

### 8.3 Out-of-Sample Performance

A related concern is **look-ahead bias**: factors identified using the full history of data cannot be tested out-of-sample on the same data. McLean and Pontiff (2016) showed that anomalies identified in academic research have significantly weaker performance in the post-publication period — consistent with a combination of overfitting and arbitrage by informed investors who trade on published findings.

The decay in factor premia after publication has two non-exclusive explanations:
1. **Statistical overfitting**: The in-sample premium was partly spurious; the out-of-sample premium is lower.
2. **Arbitrage**: Informed investors trade the anomaly, reducing the mispricing. If the premium is real, it should persist (risk-based); if it is mispricing, it should decay.

### 8.4 What Constitutes a Valid Factor?

Cochrane (2011) argued that identifying which factors are genuine requires:

1. **Economic theory**: The factor should have a clear theoretical motivation as a priced risk or an SDF ingredient.
2. **Out-of-sample evidence**: The factor should work in international data, in different time periods, and in asset classes other than equities.
3. **Spanning tests**: The factor should not be spanned by existing factors — it adds information beyond what is already in the model.
4. **Conservative $t$-statistics**: The $t$-statistic should exceed 3.0 (Harvey et al.) or be robust to reasonable resampling.

By these criteria, only a handful of factors survive: the market, value, profitability, and momentum have the strongest evidence. Investment has mixed evidence. Most of the remaining 300+ factors are unlikely to be genuine priced risks.

In [ ]:
# Simulate the multiple testing problem
# Show: how many factors appear significant purely by chance?

n_factors_tested = 300   # number of candidate factors tested
T_mt = 600               # 50 years monthly observations
n_simulations = 2000     # Monte Carlo replications

# Under H0: ALL factors are null (no true risk premium)
# Generate factor return series: iid N(0, sigma_factor)
sigma_factor_mt = 0.035  # monthly factor vol

# For each simulation, count how many factors have |t| > 2.0 and |t| > 3.0
count_t20 = np.empty(n_simulations)  # |t| > 2.0
count_t30 = np.empty(n_simulations)  # |t| > 3.0

for sim in range(n_simulations):
    # Draw n_factors_tested factors of length T_mt
    F_null = rng.normal(0, sigma_factor_mt, (T_mt, n_factors_tested))
    # t-statistic for each factor: t = sqrt(T) * mean(F) / std(F)
    t_stats = np.sqrt(T_mt) * F_null.mean(axis=0) / F_null.std(axis=0)
    count_t20[sim] = np.sum(np.abs(t_stats) > 2.0)
    count_t30[sim] = np.sum(np.abs(t_stats) > 3.0)

# Under H0: expected false positives
from scipy.stats import norm as norm_stats
expected_t20 = n_factors_tested * 2 * (1 - norm_stats.cdf(2.0))
expected_t30 = n_factors_tested * 2 * (1 - norm_stats.cdf(3.0))

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

ax = axes[0]
ax.hist(count_t20, bins=40, density=True, alpha=0.7, color='tomato',
        label=f'|t|>2.0: mean={count_t20.mean():.1f}')
ax.hist(count_t30, bins=20, density=True, alpha=0.7, color='steelblue',
        label=f'|t|>3.0: mean={count_t30.mean():.1f}')
ax.axvline(expected_t20, color='tomato', ls='--', lw=2,
           label=f'Expected t>2: {expected_t20:.1f}')
ax.axvline(expected_t30, color='steelblue', ls='--', lw=2,
           label=f'Expected t>3: {expected_t30:.1f}')
ax.set_xlabel('Number of factors with significant t-statistic')
ax.set_ylabel('Density')
ax.set_title(f'Multiple Testing Simulation\n({n_factors_tested} null factors, T={T_mt} months)')
ax.legend(fontsize=9)

# Panel 2: illustrate the Bonferroni and BHY corrections
alpha_fdr = 0.05
n_tests_range = np.arange(1, 500)
t_bonferroni = norm_stats.ppf(1 - alpha_fdr / (2 * n_tests_range))   # Bonferroni
t_nominal    = np.full_like(n_tests_range, norm_stats.ppf(1 - alpha_fdr/2), dtype=float)
# Harvey-Liu-Zhu (2016) approximate threshold
t_hlz = 0.7071 * np.sqrt(norm_stats.ppf(1 - alpha_fdr / (2 * n_tests_range))**2 + np.log(n_tests_range))

ax2 = axes[1]
ax2.plot(n_tests_range, t_nominal, 'grey', lw=2, ls='--',
         label='Nominal (t=1.96 for α=5%)')
ax2.plot(n_tests_range, t_bonferroni, 'tomato', lw=2,
         label='Bonferroni correction')
ax2.plot(n_tests_range, np.clip(t_hlz, 2, 5), 'steelblue', lw=2,
         label='Harvey-Liu-Zhu (2016)')
ax2.axvline(316, color='black', ls=':', lw=1.5, label='316 factors tested (HLZ)')
ax2.axhline(3.0, color='#27ae60', ls='-.', lw=1.5, label='t=3.0 threshold')
ax2.set_xlabel('Number of factors tested')
ax2.set_ylabel('Required t-statistic for significance')
ax2.set_title('Adjusted Significance Thresholds\nfor Multiple Hypothesis Testing')
ax2.set_ylim(1.5, 5)
ax2.set_xlim(1, 400)
ax2.legend(fontsize=9)

plt.tight_layout()
plt.show()

print(f"Under H0 (no true factor), testing {n_factors_tested} factors with |t|>2.0:")
print(f"  Expected false positives: {expected_t20:.1f}")
print(f"  Simulated mean:          {count_t20.mean():.1f}")
print(f"\nWith the stricter |t|>3.0 threshold:")
print(f"  Expected false positives: {expected_t30:.2f}")
print(f"  Simulated mean:          {count_t30.mean():.2f}")
print(f"\nHarvey-Liu-Zhu (2016): with 316 factors, the threshold should be approximately 3.0.")

---

## 9. Factor Spanning and Model Comparison

### 9.1 The Spanning Test

A proposed new factor $F_{new}$ is **spanned** by existing factors $\mathbf{F}$ if its alpha in a time-series regression on $\mathbf{F}$ is zero:
$$
F_{new,t} = \alpha + \boldsymbol{\beta}' \mathbf{F}_t + \varepsilon_t, \quad \alpha = 0.
$$
If $\alpha = 0$, the new factor does not improve the maximum Sharpe ratio achievable from the existing factors. It is redundant.

**Example**: Fama and French (2015) showed that HML is largely spanned by the other four factors (especially CMA and RMW) for many test portfolios. This raised the question of whether the five-factor model could be reduced to four factors without loss of pricing power.

### 9.2 Cross-Sectional R-Squared

A simple descriptive measure of model fit is the cross-sectional $R^2$ from the Fama-MacBeth second-pass regression. It measures the fraction of the cross-sectional variation in mean returns explained by the factor betas:
$$
R^2_{CS} = 1 - \frac{\sum_i (\bar{R}^i - \hat{\lambda}_0 - \hat{\boldsymbol{\lambda}}' \hat{\boldsymbol{\beta}}_i)^2}{\sum_i (\bar{R}^i - \bar{R})^2}.
$$
The CAPM achieves $R^2_{CS} \approx 0.01$ on 25 size-value portfolios (essentially zero — betas have no cross-sectional explanatory power). FF3 achieves $R^2_{CS} \approx 0.80$. FF5 achieves $R^2_{CS} \approx 0.90$.

However, the cross-sectional $R^2$ can be artificially inflated by including irrelevant factors. It should be interpreted alongside the GRS test and the HJ distance.

In [ ]:
# Spanning test: is a new factor redundant given existing ones?
# Compare model pricing performance: CAPM vs FF3 vs FF5 (cross-sectional R2)

# Fama-MacBeth on the 25 simulated portfolios
def fama_macbeth(R_test, F_factors, T_first_pass):
    """
    Two-pass Fama-MacBeth regression.
    Returns: lambda estimates, CS R2, mean |alpha|
    """
    T, N = R_test.shape
    K = F_factors.shape[1]

    # Pass 1: estimate betas in first T_first_pass periods
    X_ts = np.column_stack([np.ones(T_first_pass), F_factors[:T_first_pass]])
    betas_hat = np.empty((N, K))
    for i in range(N):
        coef = np.linalg.lstsq(X_ts, R_test[:T_first_pass, i], rcond=None)[0]
        betas_hat[i] = coef[1:]

    # Pass 2: cross-sectional regression each period in second half
    X_cs = np.column_stack([np.ones(N), betas_hat])
    T2 = T - T_first_pass
    lambdas_t = np.empty((T2, K + 1))
    for t in range(T2):
        lam = np.linalg.lstsq(X_cs, R_test[T_first_pass + t], rcond=None)[0]
        lambdas_t[t] = lam

    lam_mean = lambdas_t.mean(axis=0)
    lam_se   = lambdas_t.std(axis=0) / np.sqrt(T2)
    t_stats  = lam_mean / lam_se

    # Cross-sectional R2
    R_bar = R_test[T_first_pass:].mean(axis=0)  # mean returns in second half
    R_hat = X_cs @ lam_mean  # fitted
    ss_res = np.sum((R_bar - R_hat)**2)
    ss_tot = np.sum((R_bar - R_bar.mean())**2)
    cs_r2  = 1 - ss_res / ss_tot
    mae    = np.abs(R_bar - R_hat).mean() * 1200

    return {
        'lambda_mean': lam_mean * 1200,   # annualised
        'lambda_se':   lam_se * 1200,
        't_stats':     t_stats,
        'cs_r2':       cs_r2,
        'mae':         mae,
        'R_bar':       R_bar * 1200,
        'R_hat':       R_hat * 1200,
        'betas':       betas_hat,
    }

T_first = T_grs // 2
fm_capm = fama_macbeth(R_grs, F_grs[:, [0]],       T_first)
fm_ff3  = fama_macbeth(R_grs, F_grs[:, [0,1,2]],  T_first)
fm_ff5  = fama_macbeth(R_grs, F_grs,              T_first)

print("=" * 65)
print("Fama-MacBeth Cross-Sectional Results (annualised %)")
print("=" * 65)
for name, fm, factor_subset in [('CAPM', fm_capm, ['MKT']),
                                   ('FF3',  fm_ff3,  ['MKT','SMB','HML']),
                                   ('FF5',  fm_ff5,  factor_names)]:
    print(f"\n{name}:")
    print(f"  Cross-sectional R²: {fm['cs_r2']:.3f}")
    print(f"  Mean abs error:     {fm['mae']:.3f}% p.a.")
    labs = ['Intercept'] + factor_subset
    for lab, lam, se, t in zip(labs, fm['lambda_mean'], fm['lambda_se'], fm['t_stats']):
        sig = '***' if abs(t) > 3 else ('**' if abs(t) > 2 else '')
        print(f"  λ_{lab:6s} = {lam:6.2f}%  (t = {t:5.2f}) {sig}")

# Visualise cross-sectional fit
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, (name, fm) in zip(axes, [('CAPM', fm_capm), ('FF3', fm_ff3), ('FF5', fm_ff5)]):
    mn, mx = min(fm['R_bar'].min(), fm['R_hat'].min()) - 0.5, \
             max(fm['R_bar'].max(), fm['R_hat'].max()) + 0.5
    ax.scatter(fm['R_hat'], fm['R_bar'],
               c=size_idx, cmap='Reds', s=80, edgecolors='black', lw=0.4, zorder=5)
    ax.plot([mn, mx], [mn, mx], 'k--', lw=1.5)
    ax.set_xlabel('Predicted mean return (% p.a.)')
    ax.set_ylabel('Realised mean return (% p.a.)')
    ax.set_title(f'{name}\n$R^2_{{CS}}$={fm["cs_r2"]:.2f}, MAE={fm["mae"]:.2f}% p.a.')

plt.tight_layout()
plt.suptitle('Cross-Sectional Fit: 25 Size-Value Portfolios\n'
             '(color shade = size quintile, dark=small)',
             fontsize=12, y=1.02)
plt.show()

---

## 10. The Momentum Factor: An Anomaly That Resists Explanation

### 10.1 The Momentum Phenomenon

Jegadeesh and Titman (1993) documented that stocks with high returns over the past 3–12 months continue to outperform stocks with low past returns over the next 3–12 months. This **momentum effect** is one of the strongest and most pervasive anomalies in finance.

The standard construction of the **UMD** (Up Minus Down, or WML — Winners Minus Losers) factor:
1. Each month, rank stocks by their 12-month cumulative return, excluding the most recent month (the "12-1" return to avoid short-term reversal).
2. Go long the top 30% (winners) and short the bottom 30% (losers).
3. The return on this zero-cost portfolio is UMD.

Historical properties of UMD (US, 1963–2023, approximate):
- Mean return: ~7.5% per year.
- Standard deviation: ~16% per year.
- Sharpe ratio: ~0.47.
- **Correlation with MKT, SMB, HML: near zero** — momentum is not spanned by the Fama-French factors.

### 10.2 Why Momentum Is Puzzling

The momentum premium is puzzling for two distinct reasons:

**Reason 1 — Risk-based explanation is difficult**: Any risk-based explanation requires that winner portfolios have higher SDF exposure in bad times than loser portfolios. But momentum portfolios are highly dynamic: last year's winners become this year's losers over a 2–5 year horizon (long-run reversal, DeBondt-Thaler 1985). It is hard to construct a stable risk story for a factor that reverses sign over medium horizons.

**Reason 2 — Momentum crashes**: The momentum portfolio exhibits extreme **negative skewness** and periodic **crashes**. In market rebounds following steep declines, past losers (which have fallen furthest and have the highest beta to a rebound) outperform past winners dramatically. The most severe momentum crash occurred in 2009: the UMD factor returned approximately -83% in three months (March-May 2009). These crashes are inconsistent with a risk-based explanation under standard preferences — if momentum is a risk factor, it should pay off in bad times, not crash precisely when the market recovers.

**Behavioural explanations**: Daniel, Hirshleifer, and Subrahmanyam (1998) proposed that momentum arises from investor overconfidence — investors overreact to their own private signals, driving up prices of recent winners. The overreaction eventually reverses (long-run mean reversion). Barberis, Shleifer, and Vishny (1998) proposed underreaction to public earnings news as the mechanism.

Momentum remains the most significant unexplained anomaly in the cross-section of expected returns.

In [ ]:
# Simulate momentum-like returns and illustrate the crash risk
# Use a regime-switching model: normal regime + crash regime

T_mom = 12 * 50   # 50 years monthly

# Parameters calibrated to approximate UMD stylised facts
mu_mom_normal  = 0.008    # monthly mean in normal regime (~9.6% p.a.)
mu_mom_crash   = -0.15    # monthly mean in crash regime (-15% per month)
sigma_mom_normal = 0.04   # monthly vol in normal regime
sigma_mom_crash  = 0.12   # monthly vol in crash regime
p_crash = 0.005           # probability of being in crash regime each month
p_recover = 0.30          # probability of leaving crash regime each month

# Simulate regime
regime = np.zeros(T_mom, dtype=int)  # 0 = normal, 1 = crash
for t in range(1, T_mom):
    if regime[t-1] == 0:
        regime[t] = int(rng.random() < p_crash)
    else:
        regime[t] = int(rng.random() > p_recover)

# Draw returns
eps_mom = rng.standard_normal(T_mom)
ret_mom = np.where(regime == 0,
                   mu_mom_normal + sigma_mom_normal * eps_mom,
                   mu_mom_crash  + sigma_mom_crash  * eps_mom)

# Cumulative return
cum_mom = np.cumsum(ret_mom)
years_mom = np.arange(T_mom) / 12

fig, axes = plt.subplots(2, 2, figsize=(14, 9))

ax = axes[0, 0]
ax.plot(years_mom, cum_mom * 100, color='#27ae60', lw=1.2)
# Shade crash periods
crash_periods = regime == 1
for t in range(T_mom):
    if crash_periods[t]:
        ax.axvspan(years_mom[t], years_mom[t] + 1/12, alpha=0.25, color='red')
ax.set_xlabel('Year'); ax.set_ylabel('Cumulative return (pp)')
ax.set_title('Simulated Momentum Factor (UMD)\nRed bands = crash regime')

ax2 = axes[0, 1]
ax2.hist(ret_mom * 100, bins=80, density=True, color='#27ae60', alpha=0.75, edgecolor='none')
# Normal density for comparison
x_grid = np.linspace(-45, 25, 300)
normal_pdf = norm.pdf(x_grid, ret_mom.mean() * 100, ret_mom.std() * 100)
ax2.plot(x_grid, normal_pdf, 'k--', lw=2, label='Normal approximation')
ax2.set_xlabel('Monthly return (%)')
ax2.set_ylabel('Density')
ax2.set_title('Return Distribution\nHeavy left tail (crash risk)')
ax2.set_xlim(-45, 25)

# Compute moments
from scipy.stats import skew, kurtosis
mu_ann  = ret_mom.mean() * 1200
sig_ann = ret_mom.std() * np.sqrt(12) * 100
sk      = skew(ret_mom)
ex_kurt = kurtosis(ret_mom)  # excess kurtosis
ax2.text(0.05, 0.90, f'Mean: {mu_ann:.1f}% p.a.\nVol: {sig_ann:.1f}% p.a.\n'
         f'Skew: {sk:.2f}\nEx.Kurt: {ex_kurt:.2f}',
         transform=ax2.transAxes, fontsize=9,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
ax2.legend()

ax3 = axes[1, 0]
# Rolling Sharpe ratio (24-month window)
window = 24
rolling_sr = np.array([
    (ret_mom[t:t+window].mean() / ret_mom[t:t+window].std()) * np.sqrt(12)
    for t in range(T_mom - window)
])
ax3.plot(years_mom[window:], rolling_sr, color='#8e44ad', lw=1)
ax3.axhline(0, color='black', lw=0.8, ls='--')
ax3.axhline(mu_ann / sig_ann, color='grey', lw=1, ls=':',
            label=f'Full-sample SR = {mu_ann/sig_ann:.2f}')
ax3.set_xlabel('Year'); ax3.set_ylabel('Rolling Sharpe ratio (annualised)')
ax3.set_title('Rolling 24-Month Sharpe Ratio\nExtreme time variation — not a stable risk factor')
ax3.legend()

ax4 = axes[1, 1]
# Drawdown profile
cum_level = np.exp(cum_mom)
running_max = np.maximum.accumulate(cum_level)
drawdown = (cum_level - running_max) / running_max * 100
ax4.fill_between(years_mom, drawdown, 0, color='tomato', alpha=0.6)
ax4.set_xlabel('Year'); ax4.set_ylabel('Drawdown (%)')
ax4.set_title('Drawdown Profile\nMomentum crashes can exceed 50%')

plt.tight_layout()
plt.suptitle('Momentum Factor: High Mean Return but Crash Risk',
             fontsize=13, fontweight='bold', y=1.01)
plt.show()

print(f"Momentum annual moments:")
print(f"  Mean:          {mu_ann:.1f}%")
print(f"  Vol:           {sig_ann:.1f}%")
print(f"  Sharpe:        {mu_ann/sig_ann:.2f}")
print(f"  Skewness:      {sk:.2f}  (negative = crash risk)")
print(f"  Excess kurtosis: {ex_kurt:.2f}  (fat tails)")
print(f"\nFraction of months in crash regime: {regime.mean()*100:.1f}%")
print(f"Worst single-month return:          {ret_mom.min()*100:.1f}%")

---

## 11. Summary and Looking Ahead

This week covered the quantitative frontier of equilibrium models and the empirical structure of expected returns.

**Part A — Long-Run Risks**: The Bansal-Yaron (2004) model introduces a small but persistent component $x_t$ in consumption growth and stochastic volatility $\sigma^2_t$. Combined with Epstein-Zin preferences ($\psi > 1$, $\gamma \approx 10$), this generates:
- A large equity premium through the long-run risk channel: agents with $\psi > 1$ strongly dislike news about *future* consumption, and this sensitivity amplifies the SDF.
- A reasonable riskfree rate because $\psi > 1$ decouples the riskfree rate from risk aversion.
- Time-varying risk premia via stochastic volatility, with countercyclical premia.
- The critical condition: $A_1 > 0$ (good news raises the price-consumption ratio) requires $\psi > 1$.

**Part B — Disaster Risk**: Barro (2006) shows that a 1.7% annual probability of a 22%+ consumption contraction (calibrated from 35 countries' 20th-century history) generates a 4–6% equity premium at $\gamma \approx 3$–4 — far more modest than the CCAPM requires. The disaster premium dominates the normal-times premium. The peso problem — disasters are rare in the US postwar sample — means the model cannot be directly rejected by US data alone.

**Part C — Factor Models**: The APT (Ross, 1976) provides a no-arbitrage foundation: in a factor return structure with diversified idiosyncratic risk, expected returns must be linear in factor betas. The SDF is correspondingly linear in the factors. The Fama-French three-factor (1993) and five-factor (2015) models are the standard empirical implementations, dramatically improving on the CAPM's cross-sectional fit. The GRS test provides a formal joint test of all pricing errors. The factor zoo problem — over 300 proposed factors in the literature — requires multiple-testing corrections (Harvey-Liu-Zhu threshold of $t \approx 3.0$) and out-of-sample validation. Momentum (Jegadeesh-Titman, 1993) remains the most prominent anomaly: large premium, crash risk, and no convincing risk-based explanation.

**In Week 4**, we examine the empirical failures of factor models in more depth: the anomalies that challenge the Fama-French models (low volatility, quality, betting-against-beta), the term structure of risk premia, and the time-series predictability of equity returns.

---

## Problem Set — Week 3

**Problem 1 — Long-Run Risks and the IES**: In the Bansal-Yaron model, the coefficient $A_1$ (loading of the log price-consumption ratio on the long-run component $x_t$) is:
$$
A_1 = \frac{1 - 1/\psi}{1 - \kappa_1 \rho}.
$$
(a) Show that $A_1 > 0$ if and only if $\psi > 1$. (b) Explain why the sign of $A_1$ determines whether news about future consumption growth raises or lowers equity prices. (c) With $\psi = 0.5$, $\rho = 0.979$, and $\kappa_1 = 0.997$, compute $A_1$ and interpret the sign. (d) How does the persistence $\rho$ amplify the magnitude of $A_1$?

**Problem 2 — Disaster Risk Calibration**: Using the disaster moments from Section 3 (p = 0.017, log-uniform $b \in [0.15, 0.64]$, $\phi = 3$): (a) Compute the disaster risk premium analytically for $\gamma \in \{2, 4, 6\}$. (b) What leverage $\phi$ is needed to generate a 6% equity premium at $\gamma = 3$? (c) How sensitive is the equity premium to the disaster probability $p$? Compute $\partial (\text{EP}) / \partial p$ at $p = 0.017$, $\gamma = 3$.

**Problem 3 — The APT spanning test**: You observe four factors: $F_1$ (market), $F_2$ (size), $F_3$ (value), $F_4$ (momentum). Run the time-series regression $F_4 = \alpha + \beta_1 F_1 + \beta_2 F_2 + \beta_3 F_3 + \varepsilon$ and test $H_0: \alpha = 0$. (a) If $\alpha = 0$, what does this imply about the SDF representation of the four-factor model? (b) Generate simulated factor returns with the correlation structure from Section 6 (but adding a momentum factor with near-zero correlation to FF5 factors). What is the $t$-statistic on $\alpha$ in the spanning regression?

**Problem 4 — GRS Test Power**: Using the simulation from Section 7, (a) compute the power of the GRS test against the alternative that CAPM alphas are as large as estimated from the simulation. (b) How many years of monthly data are needed to achieve 80% power against the same alternative? (c) Discuss why the GRS test may reject the true model when the number of test portfolios $N$ is large relative to $T$.

**Problem 5 — Multiple Testing Correction**: Suppose a researcher tests 50 factors and finds 8 with $|t| > 2.0$. (a) Compute the expected number of false positives under the null using the Bonferroni correction. (b) What is the Bonferroni-adjusted significance threshold for each individual test at a 5% family-wise error rate? (c) Under the Harvey-Liu-Zhu framework, what $t$-statistic threshold should the researcher apply? (d) How many of the 8 "significant" factors likely survive this threshold?

**Problem 6 — Momentum and Crash Risk**: The momentum factor UMD has approximately the following monthly return distribution: normal-times regime (probability 0.99) with mean 0.8% and volatility 4%, and crash regime (probability 0.01) with mean -15% and volatility 12%. (a) Compute the unconditional mean, variance, skewness, and excess kurtosis of monthly UMD returns analytically. (b) Compute the unconditional Sharpe ratio (annualised). (c) A risk-averse investor with $\gamma = 5$ and CRRA utility considers allocating to UMD. Compute the certainty-equivalent improvement from holding the optimal allocation in UMD alongside a riskfree asset, relative to holding the riskfree asset alone. (d) Does the skewness of UMD make it more or less attractive than a normally distributed factor with the same mean and variance? Explain.

---

## References

- Bansal, R. and Yaron, A. (2004). Risks for the long run: A potential resolution of asset pricing puzzles. *Journal of Finance*, 59(4), 1481–1509.
- Rietz, T. A. (1988). The equity risk premium: A solution. *Journal of Monetary Economics*, 22(1), 117–131.
- Barro, R. J. (2006). Rare disasters and asset markets in the twentieth century. *Quarterly Journal of Economics*, 121(3), 823–866.
- Nakamura, E., Steinsson, J., Barro, R. and Ursua, J. (2013). Crises and recoveries in an empirical model of consumption disasters. *American Economic Journal: Macroeconomics*, 5(3), 35–74.
- Ross, S. A. (1976). The arbitrage theory of capital asset pricing. *Journal of Economic Theory*, 13(3), 341–360.
- Fama, E. F. and French, K. R. (1993). Common risk factors in the returns on stocks and bonds. *Journal of Financial Economics*, 33(1), 3–56.
- Fama, E. F. and French, K. R. (2015). A five-factor asset pricing model. *Journal of Financial Economics*, 116(1), 1–22.
- Jegadeesh, N. and Titman, S. (1993). Returns to buying winners and selling losers: Implications for stock market efficiency. *Journal of Finance*, 48(1), 65–91.
- Harvey, C. R., Liu, Y. and Zhu, H. (2016). ... and the cross-section of expected returns. *Review of Financial Studies*, 29(1), 5–68.
- Gibbons, M. R., Ross, S. A. and Shanken, J. (1989). A test of the efficiency of a given portfolio. *Econometrica*, 57(5), 1121–1152.
- McLean, R. D. and Pontiff, J. (2016). Does academic research destroy stock return predictability? *Journal of Finance*, 71(1), 5–32.
- Cochrane, J. H. (2011). Presidential address: Discount rates. *Journal of Finance*, 66(4), 1047–1108.
- Gabaix, X. (2012). Variable rare disasters: An exactly solved framework for ten puzzles in macro-finance. *Quarterly Journal of Economics*, 127(2), 645–700.
- Bollerslev, T., Tauchen, G. and Zhou, H. (2009). Expected stock returns and variance risk premia. *Review of Financial Studies*, 22(11), 4463–4492.